In [ ]:
# 예제 2: 데이터 분석 어시스턴트 - CSV/데이터 분석 Agent
# Google Colab에서 실행하세요

# ============================
# 1. 필요한 패키지 설치
# ============================
!pip -q install langchain langchain-openai langchain-community yfinance gradio matplotlib



In [4]:

# ============================
# 2. 라이브러리 임포트 및 설정
# ============================
import yfinance as yf
import matplotlib.pyplot as plt
import gradio as gr
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage


In [5]:


# ============================
# 3. API 키 설정 (Colab Secrets 또는 직접 입력)
# ============================
import os

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "RAG_Agent_3"                 # 수업용 프로젝트명

In [ ]:
# ============================================
# 4. 해외 주식 데이터 수집 Tool 정의
# ============================================
def get_stock_data(ticker: str, period: str = "1mo") -> str:
    """
    해외 주식 데이터 (Yahoo Finance 기반)
    ticker 예시: AAPL, TSLA, MSFT, NVDA, AMZN, GOOG
    period 예시: '5d','1mo','3mo','6mo','1y','5y','max'
    """
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        if hist.empty:
            return f"{ticker} 데이터를 가져올 수 없습니다."

        latest = hist.tail(1)
        close_price = latest["Close"].values[0]
        return f"{ticker} 최근 종가: {close_price:.2f} USD (기간: {period})"
    except Exception as e:
        return f"에러 발생: {e}"

stock_tool = Tool(
    name="get_stock_data",
    description="해외 주식 티커(ticker)와 기간(period)을 입력받아 최근 주가 정보를 요약합니다.",
    func=get_stock_data
)

# ============================================
# 5. Agent 설정
# ============================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [stock_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 해외 주식 분석 어시스턴트이다.
사용자가 요청하면 주식 데이터를 가져오고 간단한 분석을 제공한다."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ============================================
# 6. Gradio용 함수 정의
# ============================================
def process_query(message, history):
    chat_history = []
    if history:
        for h in history:
            if h[0]: chat_history.append(HumanMessage(content=h[0]))
            if h[1]: chat_history.append(AIMessage(content=h[1]))

    response = agent_executor.invoke({
        "input": message,
        "chat_history": chat_history
    })
    return response["output"]

def plot_stock(ticker: str, period: str = "1mo"):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        if hist.empty:
            return None, f"{ticker} 데이터 없음"

        plt.figure(figsize=(6,4))
        plt.plot(hist.index, hist["Close"], label="Close Price")
        plt.title(f"{ticker} ({period})")
        plt.xlabel("Date")
        plt.ylabel("Price (USD)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        img_path = f"{ticker}_{period}.png"
        plt.savefig(img_path)
        plt.close()
        return img_path, f"{ticker} 주가 그래프 ({period})"
    except Exception as e:
        return None, f"그래프 생성 실패: {e}"

# ============================================
# 7. Gradio 인터페이스
# ============================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📈 해외 주식 분석 챗봇\nLangChain + Agent + Gradio + Yahoo Finance")

    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="질문 입력", placeholder="예: 테슬라(TSLA) 최근 1개월 주가 알려줘")
    with gr.Row():
        ticker_box = gr.Textbox(label="주식 티커", value="TSLA")
        period_box = gr.Dropdown(
            ["5d","1mo","3mo","6mo","1y"], value="1mo", label="기간 선택"
        )
    graph = gr.Image(label="주가 그래프")
    info = gr.Textbox(label="분석 결과")

    def respond(message, chat_history, ticker, period):
        agent_answer = process_query(message, chat_history)
        img, desc = plot_stock(ticker, period)
        chat_history.append((message, agent_answer))
        return "", chat_history, img, desc

    msg.submit(respond, [msg, chatbot, ticker_box, period_box], [msg, chatbot, graph, info])

# ============================================
# 8. 실행
# ============================================
if __name__ == "__main__":
    demo.launch(share=True, debug=True)